In [1]:
# import statsapi as mlb
# from datetime import datetime
# import time

import pandas as pd
import numpy as np
import plotly

import sql_alchemy_mapping

db=sql_alchemy_mapping.db

session = sql_alchemy_mapping.session
Base = sql_alchemy_mapping.Base

Engine(sqlite://///Users/schlinkertc/code/MLB/mlb_sqlite/mlb.db)


In [2]:
session.execute("""select count(*) from games""").fetchall()

[(194,)]

In [3]:
query=(
        """
        select 
            pitch.type_description,
            pitch.play_id,
            play.pitcher_id,
            pitch.startSpeed,
            pitch.endSpeed,
            pitch.zone,
            pitch.spinRate,
            pitch.breakLength
        from 
            pitches pitch 
            inner join plays play 
            on 
            pitch.play_id=play.id
        """
)
default_dict = {99.9:np.nan,0.0:np.nan}
pitch_df=pd.read_sql_query(query,db.db_engine)
pitch_df.replace(default_dict,inplace=True)
pitch_df.dropna(inplace=True)

In [4]:
pitch_df.head()

,type_description,play_id,pitcher_id,startSpeed,endSpeed,zone,spinRate,breakLength
0,Four-Seam Fastball,2019/08/30/chamlb-atlmlb-1AB_0,608331.0,92.9,86.9,11.0,1926.0,4.8
1,Four-Seam Fastball,2019/08/30/chamlb-atlmlb-1AB_0,608331.0,94.0,87.8,13.0,2013.0,4.8
2,Four-Seam Fastball,2019/08/30/chamlb-atlmlb-1AB_0,608331.0,95.2,88.7,13.0,2059.0,4.8
3,Four-Seam Fastball,2019/08/30/chamlb-atlmlb-1AB_0,608331.0,95.3,88.5,12.0,2038.0,3.6
4,Curveball,2019/08/30/chamlb-atlmlb-1AB_0,608331.0,75.6,70.2,13.0,2765.0,14.4


In [5]:
import plotly.graph_objects as go

In [6]:
# scatter plot of pitches by spin rate and end speed
fig = go.Figure(
    data=[
        dict(
            x=pitch_df[pitch_df['type_description'] == i]['spinRate'],
            y=pitch_df[pitch_df['type_description'] == i]['endSpeed'],
            text=pitch_df[pitch_df['type_description'] == i]['pitcher_id'],
            mode='markers',
            marker={
                'size':8,
                'line':{'width':.3,'color':'white'}
            },
            name=i
        ) for i in pitch_df.type_description.unique()
    ],
    layout=
        dict(
            xaxis={'type': 'log', 'title': 'Spin Rate'},
            yaxis={'type': 'log','title': 'End Speed'},
            autosize=True,
            #margin={'1':40,'b':40,'t':10,'r':10},
            legend={'x':0,'y':1},
            hovermode='closest',
            title='2019 Pitches by End Speed and Spin Rate'
            )
)

In [13]:
fig.write_image('images/pitch_scatter.jpg')

ValueError: 
For some reason plotly.py was unable to communicate with the
local orca server process, even though the server process seems to be running.

Please review the process and connection information below:

orca status
-----------
    state: running
    executable: //anaconda3/bin/orca
    version: 1.3.0
    port: 49490
    pid: 616
    command: ['//anaconda3/bin/orca', 'serve', '-p', '49490', '--plotly', '//anaconda3/lib/python3.7/site-packages/plotly/package_data/plotly.min.js', '--graph-only', '--mathjax', 'https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.5/MathJax.js']




In [12]:
import plotly.io as pio
#pio.show(fig,renderer='iframe')
#pio.write_image(fig,'images/pitch_scatter.jpg',format='jpg')

In [22]:
import plotly.express as px

In [26]:
df = pd.read_sql('''select * from pitches''',db.db_engine)

In [27]:
df.columns

Index(['id', 'call_code', 'call_description', 'ballColor', 'trailColor',
       'isInPlay', 'isStrike', 'isBall', 'type_code', 'type_description',
       'hasReview', 'count_balls', 'count_strikes', 'startSpeed', 'endSpeed',
       'strikeZoneTop', 'strikeZoneBottom', 'aY', 'aZ', 'pfxX', 'pfxZ', 'pX',
       'pZ', 'vX0', 'vY0', 'vZ0', 'x', 'y', 'x0', 'y0', 'z0', 'aX',
       'breakAngle', 'breakLength', 'breakY', 'spinRate', 'spinDirection',
       'zone', 'typeConfidence', 'plateTime', 'extension', 'index', 'pfxId',
       'pitchId', 'startTime', 'endTime', 'play_id'],
      dtype='object')

In [28]:
df['call_description'].value_counts()

Ball                         54400
Foul                         27348
Called Strike                26330
In play, out(s)              19562
Swinging Strike              19234
In play, no out               6993
In play, run(s)               4183
Ball In Dirt                  3668
Foul Tip                      1418
Swinging Strike (Blocked)     1227
Automatic Ball                 540
Hit By Pitch                   495
Foul Bunt                      367
Missed Bunt                     76
Pitchout                        16
Name: call_description, dtype: int64

In [33]:
df[df['call_description']=='Called Strike']['count_balls'].value_counts()

0    15547
1     5890
2     2962
3     1931
Name: count_balls, dtype: int64

In [36]:
df['spinDirection'].value_counts()

99.9     14790
206.0     1523
204.0     1522
207.0     1508
205.0     1501
         ...  
0.0         40
352.0       39
358.0       33
2.0         33
353.0       29
Name: spinDirection, Length: 361, dtype: int64